In [1]:
%load_ext autoreload
%autoreload 2
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten, LSTM
import pandas as pd
import numpy as np

In [2]:
from RainPredictionMachine.data import clean_data_rpm
df = clean_data_rpm()
df = df.clean_data(1)
df.head()

,Chuva,Pres,Pres_max,Pres_min,Radiacao,Temp,Temp_orvalho,Temp_max,Temp_min,Temp_orvalho_max,...,Umid,Dir_vento,Rajada_vento,Vel_vento,Estaçao,Latitude,Longitude,Altitude,datahora,classe_chuva
0,0.0,957.9,957.9,957.1,0.0,21.7,21.7,23.7,21.7,23.7,...,100.0,130.5,3.75,2.10,TUPA,-21.927251,-50.490251,498.0,2021-01-01 00:00:00+00:00,nao chove
1,0.0,958.0,958.3,957.9,0.0,21.4,21.4,21.7,21.4,21.6,...,100.0,109.0,5.40,3.35,TUPA,-21.927251,-50.490251,498.0,2021-01-01 01:00:00+00:00,nao chove
2,0.0,957.4,958.0,957.4,0.0,21.2,21.2,21.5,21.2,21.5,...,100.0,160.5,3.20,1.40,TUPA,-21.927251,-50.490251,498.0,2021-01-01 02:00:00+00:00,nao chove
3,0.0,956.6,957.4,956.6,0.0,21.0,21.0,21.3,20.9,21.3,...,100.0,160.5,3.20,1.40,TUPA,-21.927251,-50.490251,498.0,2021-01-01 03:00:00+00:00,nao chove
4,0.0,957.0,957.0,956.6,0.0,21.3,21.3,21.3,21.0,21.3,...,100.0,160.5,3.20,1.40,TUPA,-21.927251,-50.490251,498.0,2021-01-01 04:00:00+00:00,nao chove


In [3]:
from RainPredictionMachine.Trainer import pipe_creator
X_train_t, X_test_t, y_train, y_test = pipe_creator(df)

/Users/isabelagaio/.pyenv/versions/3.8.12/envs/rain-prediction-machine/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [4]:
X_train_t.shape

(6132, 20)

In [5]:
X_test_t.shape

(2628, 20)

In [6]:
X_train_t1 = np.expand_dims(X_train_t, 2)

In [7]:
X_train_t1.shape

(6132, 20, 1)

In [8]:
X_test_t1 = np.expand_dims(X_test_t, 2)

In [9]:
X_test_t1.shape

(2628, 20, 1)

In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_pad = pad_sequences(X_train_t1)
X_pad = pad_sequences(X_train_t1, dtype= 'float32', padding= 'post', value=-1) 
X_pad.sort()

In [11]:
normalizer = Normalization()
normalizer.adapt(X_train_t)
normalizer(X_train_t)[0]

2022-03-22 13:57:52.492042: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([-0.0180628 , -0.24303786, -0.08000493,  0.25810885,  0.07446642,
        0.08463077,  0.14668016, -0.0701665 ,  0.2548198 ,  0.18735255,
        0.1518409 ,  0.236537  ,  0.13763003,  0.20665053, -0.3088882 ,
       -0.136638  , -0.2204804 ,  0.27166808,  0.49918643, -0.09999113],
      dtype=float32)>

In [12]:
norm = Normalization()

model = Sequential()

model.add(norm)

model.add(LSTM(units=20, activation='tanh'))
model.add(Dense(10, activation="tanh"))
model.add(Dense(4, activation="softmax"))

model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accurancy'])


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normalizat  (None, 20, 1)            3         
 ion)                                                            
                                                                 
 lstm (LSTM)                 (None, 20)                1760      
                                                                 
 dense (Dense)               (None, 10)                210       
                                                                 
 dense_1 (Dense)             (None, 4)                 44        
                                                                 
Total params: 2,017
Trainable params: 2,014
Non-trainable params: 3
_________________________________________________________________


In [15]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(X_train_t1, y_train, batch_size=16, epochs=1000, verbose=1,
         validation_split=0.2,
          callbacks=[es])

Epoch 1/1000


TypeError: 'NoneType' object is not callable